# Sky Simulation using SDP Imaging
#
This example mirrors the original sky simulation flow but uses the SDP backend
for imaging via `get_imager` / `ImageSpec`.



In [1]:
import math
import os
from datetime import datetime, timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord

from karabo.imaging.imager_factory import ImagingBackend, get_imager
from karabo.imaging.imager_interface import ImageSpec
from karabo.simulation.beam import generate_gaussian_beam_data
from karabo.simulation.interferometer import FilterUnits, InterferometerSimulation
from karabo.simulation.observation import Observation
from karabo.simulation.sky_model import SkyModel
from karabo.simulation.telescope import Telescope
from karabo.simulator_backend import SimulatorBackend
from karabo.util.file_handler import FileHandler

/home/desantis/miniconda3/envs/karabo/lib/python3.9/site-packages/ska_sdp_datamodels/configuration/config_create.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


## Parameters



In [2]:
RA_DEG = 20
DEC_DEG = -30
START_TIME = datetime(2000, 3, 20, 12, 6, 39)
OBS_LENGTH = timedelta(hours=3)
START_FREQ = 1.5e9
FREQ_BIN = 1.0e7
NUM_TIME_STEPS = 10
NUM_CHANNELS = 2

IMAGING_NPIXEL = 1024
CELLSIZE_RAD = 2 / 180 * math.pi / IMAGING_NPIXEL

OUTPUT_BASE_DIRECTORY = Path(
    FileHandler().get_tmp_dir(
        prefix="sky-simulation-sdp-",
        purpose="Sky simulation with SDP imaging",
    )
)

Creating /tmp/karabo-STM-desantis-kveFKedSic/sky-simulation-sdp-plLGllI9F4 for Sky simulation with SDP imaging


## Sky and Observation



In [3]:
sky = SkyModel.sky_test()
sky.explore_sky(phase_center=(RA_DEG, DEC_DEG), wcs_enabled=False)

observation = Observation(
    phase_centre_ra_deg=RA_DEG,
    phase_centre_dec_deg=DEC_DEG,
    start_date_and_time=START_TIME,
    length=OBS_LENGTH,
    number_of_time_steps=NUM_TIME_STEPS,
    start_frequency_hz=START_FREQ,
    frequency_increment_hz=FREQ_BIN,
    number_of_channels=NUM_CHANNELS,
)

simulation = InterferometerSimulation(
    channel_bandwidth_hz=FREQ_BIN,
    time_average_sec=8,
    ignore_w_components=True,
    uv_filter_units=FilterUnits.Metres,
    use_gpus=False,
    use_dask=False,
    station_type="Isotropic beam",
)

## Simulation backend and telescope



In [4]:
simulator_backend = SimulatorBackend.SDP
telescope = Telescope.constructor("MID", backend=SimulatorBackend.RASCIL)

## Run simulation (MS output)



In [5]:
visibility = simulation.run_simulation(
    telescope=telescope,
    sky=sky,
    observation=observation,
    backend=simulator_backend,
    primary_beam=None,
    visibility_format="MS",
    visibility_path=os.path.join(OUTPUT_BASE_DIRECTORY, "SDP.MS"),
)
print(f"Created visibility at {visibility.path}")

INFO: SKA-SDP could not exactly match your observation start time.
This is expected behaviour of the underlying time conversion.
There is a time difference of 23 h 17 m 29.142 s
Matched path /tmp/karabo-STM-desantis-kveFKedSic/sky-simulation-sdp-plLGllI9F4/SDP.MS to format MS
Created visibility at /tmp/karabo-STM-desantis-kveFKedSic/sky-simulation-sdp-plLGllI9F4/SDP.MS


## Imaging with SDP backend



In [6]:
imager = get_imager(ImagingBackend.SDP)
spec = ImageSpec(
    npix=IMAGING_NPIXEL,
    cellsize_arcsec=math.degrees(CELLSIZE_RAD) * 3600.0,
    phase_centre_deg=(RA_DEG, DEC_DEG),
)

dirty_image, psf_image = imager.invert(visibility, spec)

print(f"Dirty image path: {dirty_image.path}")
print(f"PSF image path:   {psf_image.path}")
print("Dirty image shape:", dirty_image.data.shape)
print("PSF image shape:", psf_image.data.shape)

Creating /tmp/karabo-STM-desantis-kveFKedSic/sdp-imager-1wSdgspyK0 for dirty and psf storage
Dirty image path: /tmp/karabo-STM-desantis-kveFKedSic/sdp-imager-1wSdgspyK0/dirty.fits
PSF image path:   /tmp/karabo-STM-desantis-kveFKedSic/sdp-imager-1wSdgspyK0/psf.fits
Dirty image shape: (1, 1, 1024, 1024)
PSF image shape: (1, 1, 1024, 1024)


## Optional quick-look plot



In [9]:
plt.imshow(dirty_image.get_squeezed_data(), cmap="inferno")
plt.colorbar()
plt.title("Dirty image (SDP)")
plt.show()